In [1]:
import pandas as pd 
import catboost as cb 
from sklearn.model_selection import train_test_split

In [20]:
train_values = pd.read_csv("train_values_catboost.csv", index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [33]:
test_values =  pd.read_csv("test_values.csv", index_col='building_id')
test_values1 =  pd.read_csv("Archivos auxiliares KNN/test_values_short1.csv", index_col='building_id')

In [3]:
importances_superestructures = [0.11087213, 0.24814224, 0.06244005, 0.03015171, 0.08679662,
       0.1561809 , 0.0585955 , 0.04264239, 0.07171384, 0.11541968,
       0.01704496]

superestructures = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other']

reemplazos = {}
for i in range(len(superestructures)):
    reemplazos[superestructures[i]] = {1:importances_superestructures[i]}    

In [4]:
test_values.replace(reemplazos, inplace=True)

In [6]:
test_values['hasnt_secondary'] = (test_values['has_secondary_use'] != True) * 1

In [7]:
importances_secondary_uses = [0.19752775, 0.27044932, 0.21866441, 0.02404361, 0.00476977,
       0.00504387, 0.00198575, 0.00234236, 0.00102254, 0.02242508,
       0.25172554]
secondary_uses =  ['has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other', 'hasnt_secondary']

reemplazos = {}
for i in range(len(secondary_uses)):
    reemplazos[secondary_uses[i]] = {1:importances_secondary_uses[i]}    

In [8]:
test_values.replace(reemplazos, inplace=True)

In [10]:
test_values ['superestructure'] = test_values\
                .loc[:, superestructures].idxmax(axis='columns')
test_values.drop(columns=superestructures, inplace=True)


test_values ['secondary_use'] = test_values\
                .loc[:, secondary_uses].idxmax(axis='columns')
test_values.drop(columns=secondary_uses, inplace=True)

In [11]:
test_values.drop(columns=["has_secondary_use"], inplace=True)
test_values.drop(columns=[ "geo_level_1_id","geo_level_2_id","geo_level_3_id"], inplace=True)

In [14]:
test_values = test_values.merge(test_values1[["min_ic", "max_ic"]],right_index=True, left_index=True)

In [15]:
test_values.to_csv('test_values_catboost.csv', index=True)

In [288]:
for col in train_values.columns:
    print(train_values[col].value_counts())

2    156623
3     55617
1     40441
4      5424
5      2246
6       209
7        39
9         1
8         1
Name: count_floors_pre_eq, dtype: int64
10     38896
15     36010
5      33697
20     32182
0      26041
25     24366
30     18028
35     10710
40     10559
50      7257
45      4711
60      3612
80      3055
55      2033
70      1975
995     1390
100     1364
65      1123
90      1085
85       847
75       512
95       414
120      180
150      142
200      106
110      100
105       89
125       37
115       21
140        9
130        9
180        7
160        6
170        6
175        5
135        5
145        3
190        3
195        2
165        2
155        1
185        1
Name: age, dtype: int64
6      42013
7      36752
5      32724
8      28445
9      22199
       ...  
82         1
80         1
78         1
75         1
100        1
Name: area_percentage, Length: 84, dtype: int64
5     78513
6     46477
4     37763
7     35465
3     25957
8     13902
2      9305
9      

In [17]:
# A excepcion de los geo_levels todas las columnas tienen pocos valores por lo que 
# se usar todas como categoricas
categoricas = []
for col in train_values.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col)
        categoricas.append(train_values.columns.get_loc(col))
        train_values[col] = train_values[col].astype("category")

In [18]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   count_floors_pre_eq     260601 non-null  category
 1   age                     260601 non-null  category
 2   area_percentage         260601 non-null  category
 3   height_percentage       260601 non-null  category
 4   land_surface_condition  260601 non-null  category
 5   foundation_type         260601 non-null  category
 6   roof_type               260601 non-null  category
 7   ground_floor_type       260601 non-null  category
 8   other_floor_type        260601 non-null  category
 9   position                260601 non-null  category
 10  plan_configuration      260601 non-null  category
 11  legal_ownership_status  260601 non-null  category
 12  count_families          260601 non-null  category
 13  superestructure         260601 non-null  category
 14 

In [28]:
model = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1', iterations=100) 

In [30]:
model.fit(train_values, train_labels, categoricas)

Learning rate set to 0.5
0:	learn: 0.7136561	total: 669ms	remaining: 1m 6s
1:	learn: 0.7256420	total: 1.27s	remaining: 1m 2s
2:	learn: 0.7291241	total: 1.84s	remaining: 59.4s
3:	learn: 0.7302839	total: 2.43s	remaining: 58.3s
4:	learn: 0.7329867	total: 3.05s	remaining: 58s
5:	learn: 0.7371779	total: 3.68s	remaining: 57.6s
6:	learn: 0.7382902	total: 4.35s	remaining: 57.8s
7:	learn: 0.7409495	total: 4.96s	remaining: 57.1s
8:	learn: 0.7418700	total: 5.64s	remaining: 57s
9:	learn: 0.7418293	total: 6.21s	remaining: 55.9s
10:	learn: 0.7420858	total: 6.83s	remaining: 55.3s
11:	learn: 0.7432882	total: 7.37s	remaining: 54s
12:	learn: 0.7443210	total: 7.93s	remaining: 53.1s
13:	learn: 0.7448991	total: 8.52s	remaining: 52.3s
14:	learn: 0.7457011	total: 9.11s	remaining: 51.6s
15:	learn: 0.7465063	total: 9.74s	remaining: 51.2s
16:	learn: 0.7471065	total: 10.3s	remaining: 50.3s
17:	learn: 0.7477434	total: 10.9s	remaining: 49.6s
18:	learn: 0.7478188	total: 11.5s	remaining: 49s
19:	learn: 0.7480291	tot

In [34]:
test_values = pd.read_csv("test_values_catboost.csv", index_col='building_id')

In [35]:
pred = model.predict(test_values) 

In [37]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission = pd.DataFrame(data=pred,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [38]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [40]:
submission.to_csv('subs/catboost_001.csv',index=True)